<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-final-PG-NO-147/blob/main/aimlrust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'supuntheekshanam'
os.environ['KAGGLE_KEY'] = '97fb6c99381c9d6779d3f904ef279dce'
clear_output()

In [ ]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip
clear_output()

In [ ]:
import tensorflow as tf
import os
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

training_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

print(class_names)

Found 48001 files belonging to 26 classes.
['Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Cherry_(including_sour)___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Apple___Apple_scab', 'Tomato___Septoria_leaf_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Apple___Black_rot', 'Squash___Powdery_mildew', 'Tomato___Early_blight', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Leaf_Mold', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Grape___Black_rot', 'Tomato___Bacterial_spot', 'Corn_(maize)___Northern_Leaf_Blight', 'Potato___Late_blight', 'Potato___Early_blight', 'Apple___Cedar_apple_rust', 'Peach___Bacterial_spot', 'Orange___Haunglongbing_(Citrus_greening)', 'Grape___Esca_(Black_Measles)']


In [ ]:
data_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

class_names = [d for d in os.listdir(data_dir)
               if os.path.isdir(os.path.join(data_dir, d)) and 'healthy' not in d.lower()]

validation_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)

Found 12000 files belonging to 26 classes.


In [ ]:
print("Classes used for training:")
print(class_names)

Classes used for training:
['Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Cherry_(including_sour)___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Apple___Apple_scab', 'Tomato___Septoria_leaf_spot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Apple___Black_rot', 'Squash___Powdery_mildew', 'Tomato___Early_blight', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Leaf_Mold', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Grape___Black_rot', 'Tomato___Bacterial_spot', 'Corn_(maize)___Northern_Leaf_Blight', 'Potato___Late_blight', 'Potato___Early_blight', 'Apple___Cedar_apple_rust', 'Peach___Bacterial_spot', 'Orange___Haunglongbing_(Citrus_greening)', 'Grape___Esca_(Black_Measles)']


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))
validation_set = validation_set.map(lambda x, y: (Rescaling(1./255)(x), y))

In [ ]:
from tensorflow.keras import layers
geometric_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

def apply_geometric_augmentation(image, label):
    image = geometric_augmentation(image)
    return image, label

training_set = training_set.map(apply_geometric_augmentation)

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
from keras.optimizers import AdamW, SGD

In [ ]:
model = Sequential([
    tf.keras.Input(shape=(224, 224, 3)),
    Conv2D(32, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(32, 3, activation='relu'),
    BatchNormalization(),
    MaxPool2D(2,2),

    Conv2D(64, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, 3, activation='relu'),
    BatchNormalization(),
    MaxPool2D(2,2),

    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPool2D(2,2),

    Conv2D(256, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, 3, activation='relu'),
    BatchNormalization(),
    MaxPool2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')
])

In [ ]:
model.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 222, 222, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 111, 111, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 111, 111, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 54, 54, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 54, 54, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 26, 26, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 26, 26, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             

 Total params: 20,064,314 (76.54 MB)

 Trainable params: 20,062,394 (76.53 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [ ]:
import sys
from io import StringIO

stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)

with open("/content/drive/MyDrive/model_summary.txt", "w") as f:
    f.write(short_model_summary)

print("Model summary saved to /content/drive/MyDrive/models/model_summary.txt")

Model summary saved to /content/drive/MyDrive/models/model_summary.txt


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

In [ ]:
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 1020s 662ms/step - accuracy: 0.4156 - loss: 2.1693 - val_accuracy: 0.5482 - val_loss: 1.6772 - learning_rate: 1.0000e-04
Epoch 2/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 975s 650ms/step - accuracy: 0.7064 - loss: 0.9880 - val_accuracy: 0.7048 - val_loss: 1.1934 - learning_rate: 1.0000e-04
Epoch 3/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 969s 645ms/step - accuracy: 0.7931 - loss: 0.7102 - val_accuracy: 0.7109 - val_loss: 1.2136 - learning_rate: 1.0000e-04
Epoch 4/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 969s 645ms/step - accuracy: 0.8306 - loss: 0.5734 - val_accuracy: 0.7487 - val_loss: 1.0826 - learning_rate: 1.0000e-04
Epoch 5/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 965s 642ms/step - accuracy: 0.8551 - loss: 0.4894 - val_accuracy: 0.6991 - val_loss: 1.5642 - learning_rate: 1.0000e-04
Epoch 6/10
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 969s 645ms/step - accuracy: 0.8808 - loss: 0.4047 - val_accuracy: 0.7841 - val_loss: 0.8702 - learning_rate: 1.0000e-04
Epoch 7/10
1500/1501 ━━━━━━

In [ ]:
train_loss,train_acc = model.evaluate(training_set)

1501/1501 ━━━━━━━━━━━━━━━━━━━━ 838s 558ms/step - accuracy: 0.9538 - loss: 0.1478


In [ ]:
val_loss,val_acc = model.evaluate(validation_set)

375/375 ━━━━━━━━━━━━━━━━━━━━ 18s 48ms/step - accuracy: 0.9112 - loss: 0.3183


In [ ]:
model.save("/content/drive/MyDrive/models/model26.keras")

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/models/model26.keras")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/drive/MyDrive/path/to/your/image.jpg'

img = image.load_img(img_path, target_size=(224, 224))

img_array = image.img_to_array(img)


img_array = np.expand_dims(img_array, axis=0)

preprocessed_image = img_array / 255.0

In [ ]:
prediction = model.predict(preprocessed_image)
prediction,prediction.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


(array([[1.9965641e-14, 2.6980317e-07, 5.8268140e-11, 8.5792884e-08,
         2.0488612e-06, 5.9247550e-06, 9.9291261e-05, 7.3190132e-04,
         6.4906466e-04, 2.6217925e-10, 2.2102043e-04, 4.0047343e-07,
         3.0101359e-07, 6.8552490e-11, 7.9553730e-10, 2.1691960e-06,
         1.3477934e-13, 5.4010257e-10, 1.3982147e-10, 6.8971198e-09,
         6.4505815e-07, 9.9828678e-01, 5.2408352e-11, 7.6869334e-08,
         2.8035230e-10, 1.8401529e-10]], dtype=float32),
 (1, 26))

In [ ]:
result_index = np.argmax(prediction)
result_index

np.int64(21)

In [ ]:
class_names[result_index]

'Potato___Early_blight'